In [12]:

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
#TransferLearning of ATTACK --> 
# VS From Scratch
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from finetune_framework import *

Files already downloaded and verified
Files already downloaded and verified


# Transfer Learning 

In [15]:
model_name = "resnet"
num_classes = 10
feature_extract = True

PyTorch Version:  1.7.1+cu110
Torchvision Version:  0.8.2+cu110


In [22]:
model_transfer, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
train_dataset, val_dataset, train_loader, val_loader = CIFAR10(128, finetune = True, 
                                                               input_size = input_size, test_batch_size=128)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"


Files already downloaded and verified
Files already downloaded and verified


In [26]:
img, label = next(iter(train_loader))
print(img.shape)

torch.Size([128, 3, 224, 224])


In [30]:
model_transfer = model_transfer.to(device)
params_to_update = model_transfer.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_transfer.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_transfer.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_transfer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [31]:
from torch.utils.tensorboard import SummaryWriter
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
PATH = './cifar_resnet_transfer.pth'
device = "cuda"
writer = SummaryWriter("Standard_Resnet_Transfer")


In [ ]:
train(model_transfer, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_transfer, 100, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 1.2100739535468314	 lr: 0.001
Epoch [1/100]	 Validation Loss: 0.8084070531627799	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 1.1801986897083194	 lr: 0.001
Epoch [2/100]	 Validation Loss: 0.7922419867938078	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 1.1553457841239012	 lr: 0.001
Epoch [3/100]	 Validation Loss: 0.7641311536861372	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 1.1383826258542287	 lr: 0.001
Epoch [4/100]	 Validation Loss: 0.7570775717119628	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 1.1390367284455263	 lr: 0.001
Epoch [5/100]	 Validation Loss: 0.7355874395823176	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 1.1333250161022177	 lr: 0.001
Epoch [6/100]	 Validation Loss: 0.7347398509707632	 lr: 0.001
Epoch [6/100]	


In [21]:
print("the clean accuracy is ")
test(val_loader, model_transfer)


the clean accuracy is 
Accuracy of the network on the 10000 test images: 46 %


In [ ]:
pgd_attack_transfer = generate_adv(model_transfer, "pgd")
fgsm_attack_transfer = generate_adv(model_transfer, "fgsm")
nifgsm_attack_transfer = generate_adv(model_transfer, "nifgsm")
vmifgsm_attack_transfer = generate_adv(model_transfer, "vmifgsm")


In [ ]:
images, labels = load_dataset(10000, val_loader)

In [ ]:
print("PGD")
adv_images_pgd_adv_t = pgd_attack_transfer(images, labels)
acc = clean_accuracy(model_transfer, adv_images_pgd_adv_t, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

In [ ]:
print("fgsm")
adv_images_fgsm_adv_t = fgsm_attack_transfer(images, labels)
acc = clean_accuracy(model_transfer, adv_images_fgsm_adv_t, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

In [ ]:
#nifgsm_attack = generate_adv(net, "nifgsm")
print("nifgsm")
adv_images_nifgsm_adv_t = nifgsm_attack_transfer(images, labels)
acc = clean_accuracy(model_transfer, adv_images_nifgsm_adv_t, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

In [ ]:
#vmifgsm_attack = generate_adv(net, "vmifgsm")
print("vmifgsm")
adv_images_vmifgsm_adv_t =vmifgsm_attack_transfer(images, labels)
acc = clean_accuracy(model_transfer, adv_images_vmifgsm_adv_t, labels)
print('[Model loaded]')
print('Acc: %2.2f %%'%(acc*100))
print("adv model")

# Adversarial Training 

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_adv = copy.deepcopy(model_transfer) #am I doing correct? 
for param in net_adv.parameters():
    print(param)

In [ ]:
params_to_update = net_adv.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in net_adv.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_transfer.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_adv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()

PATH = './cifar_trans_resnet_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_trans_resnet_PGD")
pgd_attack = generate_adv(net_adv, "pgd")

In [ ]:
train(net_adv, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_adv, 100, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

# TODO Compared with from scratch standard training and attacks 

# TODO Compared with from scratch adversarial training and attacks 